In [1]:
import pandas as pd
import numpy as n
import matplotlib.pyplot as plt
from keras.preprocessing import text as keras_text, sequence as keras_seq

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,TimeDistributed,Activation,MaxPooling2D,Embedding,RepeatVector,Dropout,Flatten
from keras.utils import to_categorical

In [3]:
data=pd.read_csv("word_split_new_test.csv")

In [4]:
data.head()

,0,0.1,1,2,3,4,5,6,7,8,...,922,923,924,925,926,927,928,929,930,931
0,"['MSG', 'SVLS', 'ARP', 'AAAT', 'NGH', 'NLLT', ...",0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"['MPN', 'ITW', 'CDL', 'PED', 'SLW', 'PGLP', 'S...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"['MVS', 'HSE', 'LRKL', 'FYS', 'ADAV', 'CFD', '...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"['MVS', 'HSE', 'LRKL', 'FCS', 'ADAV', 'CFD', '...",0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
4,"['MSK', 'FVI', 'TCI', 'AHG', 'ENL', 'PKE', 'TI...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data.shape

(9096, 933)

In [6]:
pro_lang=data["0"]



In [7]:
Tokenizer = keras_text.Tokenizer(char_level = False) #
t=Tokenizer.fit_on_texts(list(pro_lang))
list_tokenized_train = Tokenizer.texts_to_sequences(pro_lang)
X = keras_seq.pad_sequences(list_tokenized_train, maxlen=350,padding="post") #maxlen padding 1000

In [9]:
y=data.iloc[:,1:].values

In [12]:
from keras.models import load_model

In [13]:
Model=load_model("word_dense_model.hdf5")

In [53]:
y_prd=Model.predict(X)

In [41]:
np.mean(y_prd[0])

0.024845183

In [19]:
import numpy as np

In [21]:
def Make(Y_ped,threshold_define):
    Y_ped[Y_ped>=threshold_define]=1
    Y_ped[Y_ped<threshold_define]=0
    return Y_ped



In [74]:
y_temp=y_prd.copy()
Y_pred=Make(y_temp,0.18)

In [68]:
from sklearn.metrics import hamming_loss

def metrics(Y_predicted, Y_test):
    totalPrecision = 0
    totalRecall = 0
    totalF1Score = 0
    for i in range(Y_test.shape[0]):
        truePositive = 0
        trueNegative = 0
        falsePositive = 0
        falseNegative = 0
        precision = 0
        recall = 0
        for j in range(Y_test.shape[1]):
            if Y_predicted[i,j] == 1:
                if Y_test[i,j] == 1:
                    truePositive = truePositive + 1
                else:    
                    falsePositive = falsePositive + 1
            else:
                if Y_test[i,j] == 1:
                    falseNegative = falseNegative + 1
                else:
                    trueNegative = trueNegative + 1
        try:
            precision = truePositive/(truePositive + falsePositive)
        except ZeroDivisionError:
            precision = 0              
         
        totalPrecision = totalPrecision + precision
        try:
            recall = truePositive/(truePositive + falseNegative)
        except ZeroDivisionError:
            recall = 0
         
        totalRecall = totalRecall + recall
        try:
            f1Score = 2 * precision * recall / (precision + recall)
        except:
            f1Score = 0
        totalF1Score = totalF1Score + f1Score
         
    avgPrecision = totalPrecision / (Y_test.shape[0])
    avgRecall = totalRecall / (Y_test.shape[0])
    avgF1Score = totalF1Score / (Y_test.shape[0])
    #print("optimizer=Adam,epochs=300,batch_size=32")
    #print("Average Precision : " + str(avgPrecision))
    #print(avgPrecision)
    #print("Average Recall : "  + str (avgRecall))
    #print(avgRecall)
    #print("Average F1-Score : " + str(avgF1Score))
    #print(avgF1Score)
    F1Score = ( 2 * avgPrecision * avgRecall ) / ( avgPrecision + avgRecall )
    #print("F1-score : " + str(F1Score))
    hammingLoss = hamming_loss(Y_predicted, Y_test)
    #print("Hamming Loss : " + str(hammingLoss))
    return avgPrecision, avgRecall, avgF1Score, F1Score, hammingLoss
    #print(classification_report(Y_test ,Y_predicted ))

In [93]:
metrics(y_prd, y)

(0.6372878871174716,
 0.08534947924995784,
 0.13691010504725215,
 0.15053799271743187,
 0.031593167731173336)

In [75]:
from sklearn.metrics import classification_report
print(classification_report(y,Y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        46
           1       0.00      0.00      0.00        39
           2       0.00      0.00      0.00       103
           3       0.00      0.00      0.00       182
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        51
           7       0.00      0.00      0.00        51
           8       0.00      0.00      0.00       119
           9       0.00      0.00      0.00       122
          10       0.00      0.00      0.00       123
          11       0.00      0.00      0.00        59
          12       0.00      0.00      0.00        70
          13       0.00      0.00      0.00       113
          14       0.00      0.00      0.00       113
          15       0.00      0.00      0.00        84
          16       0.00      0.00      0.00       123
          17       0.00    

In [ ]:
 
###########  final result #####################    

th       p   r    f1 
0.18  0.34   0.27  0.25

